# Constants and Configuration

### Content

1. [Constants](#Const)
    1. [Hard Coded](#ConstHard)
    2. [Conigurable](#ConstConf)
    3. [Where to put constants?](#ConstWhere)
2. [Configuration](#Conf)
    1. [Config files](#ConfFiles)
    2. [Accessing configuration values](#ConfAccess)
    3. [Default Configuration](#ConfDefault)


##  1. <a id="Const"> Constants </a>
Constants are values that, once initialized, are never changed during the runtime of a program. In Python constants are assigned to variables with capital letters by convention, and vice versa, variables with capital letters are supposed to be constants.

In principle there are about four ways to define a constant's value:
- _hard coding_: the value is defined in the python code directly
- _argument_: the value is taken from an execution argument
- _context_: the value is derived from the environmental context of the exection, e.g., the current working directory or the date-time of execution start.
- _configuation_: read from a file or database

In CLIMADA, we only use _hard coding_ and _configuration_ to assign values to constants.

###  1.A. <a id='ConstHard'> Hard Coded </a>
Hard coding constants is the prefered way to deal with strings that are used to identify objects or files.

In [22]:
# suboptimal
my_dict = {'x': 4}
if my_dict['x'] > 3:
    msg = 'well, arh, ...'
msg

'well, arh, ...'

In [21]:
# good
X = 'x'
my_dict = {X: 4}
if my_dict[X] > 3:
    msg = 'yeah!'
msg

'yeah!'

In [28]:
# possibly overdoing it
X = 'x'
Y = "this doesn't mean that every string must be a constant"
my_dict = {X: 4}
if my_dict[X] > 3:
    msg = Y
msg

"this doesn't mean that every string must be a constant"

In [26]:
import pandas as pd
X = 'x'
df = pd.DataFrame({'x':[1,2,3], 'y':[4,5,6]})
try:
    df.X
except:
    from sys import stderr; stderr.write("this does not work\n")
df[X] # this does work but it's less pretty
df.x

this does not work


0    1
1    2
2    3
Name: x, dtype: int64

###  1.B. <a id='ConstConf'> Configurable </a>
When it comes to absolute pathes, it is urgently suggested to not use hard coded constant values, for the obvious reasons. But also relative pathes can cause problems. In particular, they may point to a location where the user has not sufficient access permissions. In order to avoid these problems, _all_ pathes constants in CLIMADA are supposed to be defined through configuration.\
<b style='color:darkred;font-size:100%'> &rarr; pathes must be configurable </b>

The same applies tu urls to external resources, databases or websites. Since they may change at any time, there addresses are supposed to be defined through configuration. Like this it will be possible to access them without the need of tampering with the source code or waiting for a new release.\
<b style='color:darkred;font-size:100%'> &rarr; urls must be configurable </b>

Another category of constants that should go into the configuration file are system specifications, such as number of CPU's available for CLIMADA or memory settings.\
<b style='color:darkred;font-size:100%'> &rarr; OS settings must be configurable </b>

###  1.C. <a id='ConstWhere'> Where to put constants? </a>
As a general rule, constants are defined in the module where they intrinsically belong to. If they belong equally to different modules though or they are meant to be used globally, there is the module `climada.util.constants` which is compiling constants CLIMADA wide.

##  2. <a id='Conf'> Configuration </a>

###  2.A. <a id='ConfFiles'> Configuration files </a>
The proper place to define constants that a user may want (or need) to change without changing the CLIMADA installation are the configuration files.\
These are files in _json_ format with the name `climada.conf`. There is a default config file that comes with the installation of CLIMADA. But it's possible to have several of them. In this case they are complementing one another.

CLIMADA looks for configuration files upon `import climada`. There are four loacations to look for configuration files:
- `climada/conf`, the installation directory
- `~/climada/conf`, the user's default climada directory
- `~/.config`, the user's configuration directory,
- `.`, the current working directory

At each location, the path is followed upwards until a file called `climada.conf` is found or the root of the path is reached. Hence, if e.g., `~/climada/climada.conf` is missing but `~/climada.conf` is present, the latter would be read.

When two config files are defining the same value, the priorities are:\
`[..]/./climada.conf` > `~/.config/climada.conf` > `~/climada/conf/climada.conf` > `installation_dir/climada/conf/climada.conf`

<img src="img/FileSystem-1.png" style="width:600px;">

#### Format
A configuration file is a JSON file, with the additional restriction, that all keys must be strings without a '.' (dot) character .\
For configuration valules that belong to a particular module it is suggested to reflect the code repositories file structure in the json object. For example if a configuration for `my_config_value` that blongs to the module `climada.util.dates_times` is wanted, it would be defined as 
```
{
  "util": {
    "dates_times": {
      "my_config_value": 42
    }
  }
}
```

#### Referenced Configuration Values
Configuration string values can be referenced from other configuration values. E.g.
```
{
  "a": "x",
  "b": "{a}y"
}
```
In this example "b" is eventually resolved to "xy".

###  2.B. <a id='ConfAccess'> Accessing configuration values </a>
Configuration values can be accessed through the (constant) `CONFIG` from the `climada` module:

In [1]:
from climada import CONFIG

In [3]:
CONFIG.hazard

{drought: {resources: {spei_file_url: http://digital.csic.es/bitstream/10261/153475/8}}, landslide: {resources: {opensearch: https://pmmpublisher.pps.eosdis.nasa.gov/opensearch, climatology_monthly: https://svs.gsfc.nasa.gov/vis/a000000/a004600/a004631/frames/9600x5400_16x9_30p/MonthlyClimatology/[01-12]_ClimatologyMonthly_032818_9600x5400.tif}, local_data: .}, relative_cropyield: {local_data: ~/climada/data/ISIMIP_crop}, trop_cyclone: {random_seed: 54}}

#### Data Types
The configuration itself and its attributes have the data type `climada.util.config.Config`

In [12]:
CONFIG.__class__, CONFIG.hazard.trop_cyclone.random_seed.__class__

(climada.util.config.Config, climada.util.config.Config)

The actual configuration values can be accessed as basic types (float, int, str), provided the definition is according to the respective data type:

In [4]:
CONFIG.hazard.trop_cyclone.random_seed.int()

54

In [3]:
try:
    CONFIG.hazard.trop_cyclone.random_seed.str()
except Exception as e:
    from sys import stderr; stderr.write(f"cannot convert random_seed to str: {e}\n")

cannot convert random_seed to str: <class 'int'>, not str


However, configuration string values can be converted to `pathlib.Path` objects if they are pointing to a directory.

In [19]:
CONFIG.hazard.relative_cropyield.local_data.dir()

WindowsPath('C:/Users/me/climada/data/ISIMIP_crop')

Note that converting a configuration string to a `Path` object like this will create the specified directory on the fly, unless `dir` is called with the parameter `create=False`.

### 2.C. <a id='ConfDefault'> Default Configuration </a>
The conifguration file `climada/conf/climada.conf` contains the default configuration.\
On the top level it has the following attributes
- __local\_data__: definition of main pathes for accessing and storing CLIMADA related data
    - __system__: top directory, where (persistent) climada data is stored\
      default: `~/climada/data`
    - __demo__: top directory for data that is downloaded or created in the CLIMADA tutorials\
      default: `~/climada/demo/data`
    - __save\_dir__: directory where transient (non-persistent) data is stored\
      default: `./results`
- __log\_level__: minimum log level showed by logging, one of DEBUG, INFO, WARNING, ERROR or CRITICAL.\
  default: `INFO`
- __max\_matrix\_size__: maximum matrix size that can be used, can be decreased in order to avoid memory issues\
  default: `100000000` (1e8)
- __exposures__: exposures modules specific configuration
- __hazard__: hazard modules specific configuration

In [5]:
CONFIG.__dict__.keys()

dict_keys(['_root', '_comment', 'local_data', 'exposures', 'hazard', 'log_level', 'max_matrix_size'])

#### Data Initialization
When `import climada` is executed in a python script or shell, data files from the installation directory are copied to the location specified int the current configuration.\
This happens only when climada is used for the first time with the current configuration. Subsequent execution will only check for presence of files and won't overwrite existing files.

<img src="img/FileSystem-2.png" style="width:600px;">

Thus, the home directory will automatically be populated with a climada directory and several files from the repository when climada is used.\
To prevent this and keep the home directory clean, create a config file `~/.config/climada.conf` with customized values for `local_data.system` and `local_data.demo`.\
As an example, a file with the following content would suppress creation of directories and copying of files during execution of CLIMADA code:

```
{
    "local_data": {
        "system": "/path/to/installation-dir/climada/data/system",
        "demo": "/path/to/installation-dir/climada/data/demo",
    },
}
```